In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from gensim.models import Word2Vec
from keras.preprocessing.text import text_to_word_sequence
from keras.utils import to_categorical
import inflect
import spacy
from livelossplot import PlotLossesKeras
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
from keras.callbacks import ModelCheckpoint,EarlyStopping,CSVLogger
from keras.layers import AveragePooling1D,TimeDistributed,Bidirectional
from keras.layers import MaxPooling1D,RepeatVector,Concatenate
from keras.layers import Activation,CuDNNGRU,Multiply
from keras.layers import Dropout,Flatten,Permute
from keras.layers import Reshape,Lambda,Dense
from keras.layers import Input,GRU,Dot,add
from keras.models import Model
from keras.utils import plot_model
import keras.backend as K

In [53]:
import pickle
import numba
import time

In [4]:
file_name_train =  'data/train/train_context.xlsx'

train_context = pd.read_excel(io=file_name_train)
train_context.head()

,context,contextID
0,North Carolina consists of three main geograph...,1
1,The coastal plain transitions to the Piedmont ...,2
2,The western section of the state is part of th...,3
3,The climate of the coastal plain is influenced...,4
4,The Atlantic Ocean has less influence on the c...,5


In [52]:
try:
    with open("BiDAF/train_context.txt", "rb") as myFile:
        c_corpus = pickle.load(myFile)
except:
    filters = '!\'"#$%&()*+—,-./:;<=>?@[\\]^_`{|}~\t\n'
    c_corpus = [text_to_word_sequence(train_context['context'][i],filters=filters,lower=False) for i in range(len(train_context))]
    
    with open("BiDAF/train_context.txt", "wb") as myFile:
        pickle.dump(c_corpus, myFile)

counter = len(c_corpus)

### Using spaCy to lemmatize

In [5]:
try:
    with open("BiDAF/lemmatize_context.txt", "rb") as myFile:
        c_corpus_lemm = pickle.load(myFile)

except:
    
    nlp = spacy.load("en_core_web_sm")

    @numba.jit
    def lemmatize(word_seq,nlp):
        temp = []
        for token in word_seq:
            temp.append(nlp(token)[0].lemma_)
        return temp

    c_corpus_lemm = []
    start_time = time.time()
    for i in range(len(c_corpus)):
        c_corpus_lemm.append(lemmatize(c_corpus[i], nlp))
        if i%100 == 0:
            print("Steps {0} out of {1}".format(i, len(c_corpus)))
            print("--- %s seconds ---\n" % (time.time() - start_time))
    print("Done !! ----{0}".format(time.time() - start_time))
    
    with open("BiDAF/lemmatize_context.txt", "wb") as myFile:
        pickle.dump(c_corpus_lemm, myFile)

Embedding Using GLoVe

In [6]:
try:
    with open("BiDAF/embedding.txt", "rb") as myFile:
        embeddings_dict = pickle.load(myFile)

except: 
    #Create embedding _dict that contains the vectors for each word
    embeddings_dict = {}
    count = 0
    start_time = time.time()
    with open("glove.6B/glove.6B.50d.txt", 'r', encoding="utf-8") as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], "float32")
            embeddings_dict[word] = vector
    
    with open("BiDAF/embedding.txt", "wb") as myFile:
        pickle.dump(embeddings_dict, myFile)

In [104]:
try:
    with open("BiDAF/embedded_context.txt", "rb") as myFile:
        embedded_context = pickle.load(myFile)

except:
    embedded_context = []
    missed = []
    
    for para in c_corpus:
        embedded_para = []
        for word in para:
            
            #Cap words cannot be embedded
            word = word.lower()

            # Range of values cannot be embedded (eg. '11-14'), need to convert to '11 to 14'
            try:
                embedded_para.append(embeddings_dict[word])
            except:
                if word not in missed:
                    missed.append(word)
        embedded_context.append(embedded_para)

    with open("BiDAF/embedded_context.txt", "wb") as myFile:
        pickle.dump(embedded_context, myFile)